<a href="https://cognitiveclass.ai/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDL0101ENSkillsNetwork20718188-2022-01-01"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>

<h1 align=center><font size = 5>Regression Models with Keras</font></h1>


<h3>Objective for this Notebook<h3>    
<h5> 1. Download and Clean dataset </h5>
<h5> 2. A. Build a baseline model </h5>
<h5> 3. B. Normalize the data </h5>     
<h5> 4. C. Increase the number of epochs </h5>
<h5> 5. D. Increase the number of hidden layers </h5>     

<a id="item31"></a>


## Download and Clean Dataset


Let's start by importing the <em>pandas</em> and the Numpy libraries.


In [ ]:
# All Libraries required for this lab are listed below. The libraries pre-installed on Skills Network Labs are commented. 
# If you run this notebook on a different environment, e.g. your desktop, you may need to uncomment and install certain libraries.

#!pip install numpy==1.21.4
#!pip install pandas==1.3.4
#!pip install keras==2.1.6

In [2]:
import pandas as pd
import numpy as np

<strong>The dataset is about the compressive strength of different samples of concrete based on the volumes of the different ingredients that were used to make them. Ingredients include:</strong>

<strong>1. Cement</strong>

<strong>2. Blast Furnace Slag</strong>

<strong>3. Fly Ash</strong>

<strong>4. Water</strong>

<strong>5. Superplasticizer</strong>

<strong>6. Coarse Aggregate</strong>

<strong>7. Fine Aggregate</strong>


Let's download the data and read it into a <em>pandas</em> dataframe.


In [3]:
concrete_data = pd.read_csv('https://cocl.us/concrete_data')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


So the first concrete sample has 540 cubic meter of cement, 0 cubic meter of blast furnace slag, 0 cubic meter of fly ash, 162 cubic meter of water, 2.5 cubic meter of superplaticizer, 1040 cubic meter of coarse aggregate, 676 cubic meter of fine aggregate. Such a concrete mix which is 28 days old, has a compressive strength of 79.99 MPa.


#### Let's check how many data points we have.


In [4]:
concrete_data.shape

(1030, 9)

So, there are approximately 1000 samples to train our model on. Because of the few samples, we have to be careful not to overfit the training data.


Let's check the dataset for any missing values.


In [5]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [6]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

The data looks very clean and is ready to be used to build our model.


#### Split data into predictors and target


The target variable in this problem is the concrete sample strength. Therefore, our predictors will be all the other columns.


In [7]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

<a id="item2"></a>


Let's do a quick sanity check of the predictors and the target dataframes.


In [8]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [9]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

Let's save the number of predictors to *n_cols* since we will need this number when building our network.


In [10]:
n_cols = predictors.shape[1] # number of predictors

<a id="item1"></a>


<a id='item32'></a>


## A. Building a baseline model


Let's import the packages from the Keras library that we will need to build our regression model.


In [12]:
from keras.models import Sequential
from keras.layers import Dense

<a id='item33'></a>


Let's define a function that defines our regression model for us so that we can conveniently call it to create our model.


In [13]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    #model.add(Dense(50, activation='relu'))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

The above function create a model that has one hidden layer of 10 nodes, and a ReLU activation function, using the adam optimizer and the mean squared error as the loss function.


<a id='item34'></a>


## Train and Test the Network


Randomly split the data into a training and test sets by holding 30% of the data for testing

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
X_train, X_test, y_train, y_test = train_test_split (predictors, target, test_size=0.3)

Let's call the function now to create our model.


In [18]:
# build the model
model = regression_model()

Next, we will train and test the model at the same time using the *fit* method. We will leave out 30% of the data for validation and we will train the model for 50 epochs.


In [19]:
# fit the model
model.fit(X_train, y_train, validation_split=0.3, epochs=50, verbose=2)



Train on 504 samples, validate on 217 samples
Epoch 1/50


2022-10-30 11:52:40.704181: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
2022-10-30 11:52:40.711755: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2494065000 Hz
2022-10-30 11:52:40.712949: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55afc6fd1a70 executing computations on platform Host. Devices:
2022-10-30 11:52:40.713036: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>
2022-10-30 11:52:40.831213: W tensorflow/compiler/jit/mark_for_compilation_pass.cc:1412] (One-time warning): Not using XLA:CPU for cluster because envvar TF_XLA_FLAGS=--tf_xla_cpu_global_jit was not set.  If you want XLA:CPU, either set that envvar, or use experimental_jit_scope to enable XLA:CPU.  To confirm that XLA is active, pass --vmodule=xla_compilation_cache=1 (as a proper command-line fl

 - 0s - loss: 58851.5154 - val_loss: 39193.9698
Epoch 2/50
 - 0s - loss: 26731.4375 - val_loss: 14951.2384
Epoch 3/50
 - 0s - loss: 8801.1960 - val_loss: 4067.7497
Epoch 4/50
 - 0s - loss: 2394.3920 - val_loss: 1653.6055
Epoch 5/50
 - 0s - loss: 1572.6911 - val_loss: 1567.4168
Epoch 6/50
 - 0s - loss: 1490.7120 - val_loss: 1431.3494
Epoch 7/50
 - 0s - loss: 1361.2609 - val_loss: 1338.5790
Epoch 8/50
 - 0s - loss: 1275.8605 - val_loss: 1255.5326
Epoch 9/50
 - 0s - loss: 1198.0321 - val_loss: 1175.1021
Epoch 10/50
 - 0s - loss: 1122.7563 - val_loss: 1094.8598
Epoch 11/50
 - 0s - loss: 1050.5817 - val_loss: 1022.5169
Epoch 12/50
 - 0s - loss: 984.4158 - val_loss: 955.6451
Epoch 13/50
 - 0s - loss: 916.6369 - val_loss: 888.6940
Epoch 14/50
 - 0s - loss: 855.3331 - val_loss: 826.7737
Epoch 15/50
 - 0s - loss: 796.8542 - val_loss: 768.9219
Epoch 16/50
 - 0s - loss: 742.0056 - val_loss: 713.8684
Epoch 17/50
 - 0s - loss: 690.5407 - val_loss: 662.3585
Epoch 18/50
 - 0s - loss: 643.4661 - val_l

Now, we evaluate the model on the test data and compute the mean squared error between the predicted concrete strength and the actual concrete strength.

In [20]:
model.evaluate(X_test, y_test)

309/309 [==============================] - 0s 55us/step


134.6609123748483

In [23]:
y_pred = model.predict(X_test)

In [24]:
from sklearn.metrics import mean_squared_error

In [26]:
result = mean_squared_error (y_test, y_pred)

Saving the result in a list

In [27]:
mse_list = []
mse_list.append(result)

Repeating last steps 49 more times to create a list of 50 mean squared errors.

In [32]:
for i in range(0,49):
    X_train, X_test, y_train, y_test = train_test_split (predictors, target, test_size=0.3)
    model = regression_model()
    model.fit(X_train, y_train, validation_split=0.3, epochs=50, verbose=0)
    model.evaluate(X_test, y_test)
    y_pred = model.predict(X_test)
    result = mean_squared_error (y_test, y_pred)
    mse_list.append(result)

309/309 [==============================] - 0s 63us/step


Reporting the mean and the standard deviation of the mean squared errors.

In [40]:
print(f'Mean: {np.mean(mse_list)}, Standard Deviation: {np.std(mse_list)}')

Mean: 510.98860854756936, Standard Deviation: 870.8856310191395


## B. Normalizing the data

Let's repeat Part A but using a normalized version of the data

We normalize the data by substracting the mean and dividing by the standard deviation.


In [42]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [43]:
mse_normal_list = []
for i in range(0,50):
    X_train, X_test, y_train, y_test = train_test_split (predictors_norm, target, test_size=0.3)
    model = regression_model()
    model.fit(X_train, y_train, validation_split=0.3, epochs=50, verbose=0)
    model.evaluate(X_test, y_test)
    y_pred = model.predict(X_test)
    result = mean_squared_error (y_test, y_pred)
    mse_normal_list.append(result)

309/309 [==============================] - 0s 69us/step


Reporting the mean and the standard deviation of the mean squared errors.

In [44]:
print(f'Mean: {np.mean(mse_normal_list)}, Standard Deviation: {np.std(mse_normal_list)}')

Mean: 633.1236844141929, Standard Deviation: 128.90877093089813


The mean is greater than the first time and the standard deviation is much smaller, showing that normalized data is better for the model performance.

## C. Increasing the number of epochs

Let's repeat Part B but using 100 epochs this time for training.

In [45]:
mse_C_epochs_list = []
for i in range(0,50):
    X_train, X_test, y_train, y_test = train_test_split (predictors_norm, target, test_size=0.3)
    model = regression_model()
    model.fit(X_train, y_train, validation_split=0.3, epochs=100, verbose=0)
    model.evaluate(X_test, y_test)
    y_pred = model.predict(X_test)
    result = mean_squared_error (y_test, y_pred)
    mse_C_epochs_list.append(result)

309/309 [==============================] - 0s 75us/step


Reporting the mean and the standard deviation of the mean squared errors.

In [46]:
print(f'Mean: {np.mean(mse_C_epochs_list)}, Standard Deviation: {np.std(mse_C_epochs_list)}')

Mean: 221.48667952019608, Standard Deviation: 44.82715987226672


The mean is much smaller now and the standard deviation is even smaller than before, showing that more training time made our model perform better.

## D. Increasing the number of hidden layers

Let's repeat part B but using three hidden layers of 10 nodes each and ReLU activation function.

In [47]:
# define regression model
def deeper_regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [48]:
mse_deeper_list = []
for i in range(0,50):
    X_train, X_test, y_train, y_test = train_test_split (predictors_norm, target, test_size=0.3)
    model = deeper_regression_model()
    model.fit(X_train, y_train, validation_split=0.3, epochs=50, verbose=0)
    model.evaluate(X_test, y_test)
    y_pred = model.predict(X_test)
    result = mean_squared_error (y_test, y_pred)
    mse_deeper_list.append(result)

309/309 [==============================] - 0s 103us/step


Reporting the mean and the standard deviation of the mean squared errors.

In [49]:
print(f'Mean: {np.mean(mse_deeper_list)}, Standard Deviation: {np.std(mse_deeper_list)}')

Mean: 145.30492711477035, Standard Deviation: 16.50626640610705


The mean is smaller than the last time and the standard deviation is also smaller, showing that more layers did well to the model performance even with half the training time.